# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Описание данных

Данные геологоразведки трёх регионов находятся в файлах: 

/datasets/geo_data_0.csv 

/datasets/geo_data_1.csv 

/datasets/geo_data_2.csv


id — уникальный идентификатор скважины;

f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);

product — объём запасов в скважине (тыс. баррелей).

Импортируем необходимые библиотеки для работы

In [1]:
import pandas as pd
import numpy as np
import matplotlib as mplt
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
RANDOM_STATE = RandomState(0)



pd.set_option('display.float_format', '{:,.2f}'.format)

## Загрузка и подготовка данных

Загрузим и изучим представленные данные

In [2]:
data_0 = pd.read_csv(r"C:\Users\lebed\Downloads\geo_data_0.csv")
data_1 = pd.read_csv(r"C:\Users\lebed\Downloads\geo_data_1.csv")
data_2 = pd.read_csv(r"C:\Users\lebed\Downloads\geo_data_2.csv")

In [3]:
def my_func(x):
    print('------------- Исходный датафрейм -------------')
    display(x.head())
    display(x.tail())
    print('')
    print('')
    print('------------- Общая информация о датафрейме -------------')
    print('')
    print('')
    display(x.info())
    print('------------- Количество пустых значений в датафрейме -------------')
    print('')
    print('')
    display(x.isna().sum())
    print('------------- Количество явных дубликатов в датафрейме -------------')
    display(x.duplicated().sum())
    print('')
    print('')
    print('------------- Описательная статистика -------------')
    display(x.describe())
    print('')
    print('')
    print('------------- Размер датасета -------------')
    display(x.shape)
    


my_func(data_0)
my_func(data_1)
my_func(data_2)

------------- Исходный датафрейм -------------


,id,f0,f1,f2,product
0,txEyH,0.71,-0.50,1.22,105.28
1,2acmU,1.33,-0.34,4.37,73.04
2,409Wp,1.02,0.15,1.42,85.27
3,iJLyR,-0.03,0.14,2.98,168.62
4,Xdl7t,1.99,0.16,4.75,154.04


,id,f0,f1,f2,product
99995,DLsed,0.97,0.37,6.08,110.74
99996,QKivN,1.39,-0.38,1.27,122.35
99997,3rnvd,1.03,0.02,-1.35,64.38
99998,7kl59,1.00,-0.53,1.58,74.04
99999,1CWhH,1.76,-0.27,5.72,149.63




------------- Общая информация о датафрейме -------------


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

...

0



------------- Описательная статистика -------------


,f0,f1,f2,product
count,"100,000.00","100,000.00","100,000.00","100,000.00"
mean,0.00,-0.00,2.50,95.00
std,1.73,1.73,3.47,44.75
min,-8.76,-7.08,-11.97,0.00
25%,-1.16,-1.17,0.13,59.45
50%,0.01,-0.01,2.48,94.93
75%,1.16,1.16,4.86,130.60
max,7.24,7.84,16.74,190.03




------------- Размер датасета -------------


(100000, 5)

Из полученных данных видно, что в каждом датасете 100 000 строк, 5 столбцов с одинаковыми названиями, явных дубликатов ни в одном из сете не обнаружено, данные представлены в правильном формате. Явных анамольных значений при первом рассмотрении не выявлено.

In [4]:
print(data_0.id.nunique())
print(data_1.id.nunique())
print(data_2.id.nunique())

99990
99996
99996


In [5]:
duplicates_0 = data_0[data_0['id'].duplicated()].id.tolist()
duplicates_1 = data_1[data_1['id'].duplicated()].id.tolist()
duplicates_2 = data_2[data_2['id'].duplicated()].id.tolist()

In [6]:
display(data_0[data_0['id'].isin(duplicates_0)].sort_values(by = 'id'))
display(data_1[data_1['id'].isin(duplicates_1)].sort_values(by = 'id'))
display(data_2[data_2['id'].isin(duplicates_2)].sort_values(by = 'id'))

,id,f0,f1,f2,product
66136,74z30,1.08,-0.31,6.99,127.64
64022,74z30,0.74,0.46,5.15,140.77
51970,A5aEY,-0.18,0.94,-2.09,33.02
3389,A5aEY,-0.04,0.16,0.21,89.25
69163,AGS9W,-0.93,0.12,-3.66,19.23
42529,AGS9W,1.45,-0.48,0.68,126.37
931,HZww2,0.76,0.37,1.86,30.68
7530,HZww2,1.06,-0.37,10.43,158.83
63593,QcMuo,0.64,-0.47,0.86,64.58
1949,QcMuo,0.51,-0.32,-2.22,75.50


,id,f0,f1,f2,product
5849,5ltQ6,-3.44,-12.30,2.00,57.09
84461,5ltQ6,18.21,2.19,3.99,107.81
1305,LHZR0,11.17,-1.95,3.00,80.86
41906,LHZR0,-8.99,-4.29,2.01,57.09
2721,bfPNe,-9.49,-5.46,4.01,110.99
82178,bfPNe,-6.20,-4.82,3.00,84.04
47591,wt4Uk,-9.09,-8.11,-0.00,3.18
82873,wt4Uk,10.26,-9.38,4.99,134.77


,id,f0,f1,f2,product
45404,KUPhW,0.23,-1.70,4.99,11.72
55967,KUPhW,1.21,3.18,5.54,132.83
11449,VF7Jo,2.12,-0.86,5.75,181.72
49564,VF7Jo,-0.88,0.56,0.72,136.23
44378,Vcm5J,-1.23,-2.44,1.22,137.97
95090,Vcm5J,2.59,1.99,2.48,92.33
28039,xCHr8,1.63,0.37,-2.38,6.12
43233,xCHr8,-0.85,2.10,5.60,184.39


Найдены дубликаты в трех сетах, причем по некоторым параметрам, разница в столбце product существенная, удалим дубликаты из каждого

In [7]:
data_0 = data_0.drop([66136,64022,51970,3389,69163,42529,931,7530,63593,1949,75715,21426,92341,60140,89582,97785,41724,1364,16633,90815])

In [8]:
data_0.shape

(99980, 5)

In [9]:

data_1 = data_1.drop([5849,84461,1305,41906,2721,82178,47591,82873])
data_2 = data_2.drop([45404,55967,11449,49564,44378,95090,28039,43233])

display(data_1.shape)
display(data_2.shape)

(99992, 5)

(99992, 5)

Удалили данные в data_0 20 строк, data_1 и data_2 по 8 строк

Также, удалим столбец id он будет лишним для обучения модели

In [10]:
data_0 = data_0.drop(['id'], axis=1)
data_1 = data_1.drop(['id'], axis=1)
data_2 = data_2.drop(['id'], axis=1)
display(data_0.shape)
display(data_1.shape)
display(data_2.shape)

(99980, 4)

(99992, 4)

(99992, 4)

In [11]:
display(data_0.describe())
display(data_1.describe())
display(data_2.describe())

,f0,f1,f2,product
count,"99,980.00","99,980.00","99,980.00","99,980.00"
mean,0.50,0.25,2.50,92.50
std,0.87,0.50,3.25,44.29
min,-1.41,-0.85,-12.09,0.00
25%,-0.07,-0.20,0.29,56.50
50%,0.50,0.25,2.52,91.85
75%,1.07,0.70,4.71,128.56
max,2.36,1.34,16.00,185.36


,f0,f1,f2,product
count,"99,992.00","99,992.00","99,992.00","99,992.00"
mean,1.14,-4.80,2.49,68.82
std,8.97,5.12,1.70,45.94
min,-31.61,-26.36,-0.02,0.00
25%,-6.30,-8.27,1.00,26.95
50%,1.15,-4.81,2.01,57.09
75%,8.62,-1.33,4.00,107.81
max,29.42,18.73,5.02,137.95


,f0,f1,f2,product
count,"99,992.00","99,992.00","99,992.00","99,992.00"
mean,0.00,-0.00,2.50,95.00
std,1.73,1.73,3.47,44.75
min,-8.76,-7.08,-11.97,0.00
25%,-1.16,-1.17,0.13,59.45
50%,0.01,-0.01,2.48,94.93
75%,1.16,1.16,4.86,130.59
max,7.24,7.84,16.74,190.03


Удалилив дубликаты, видим, что это практически не повлияло на общие цифры. Из данных видно, что отсутсвует взаимосвязть среднего показателя f0 - f2 от кол-ва продукта(барралей) по трем регионам, во втором регионе наблюдается по признаку f0 и f1 аномальные минимальные и максимальные значения, впревом и третьем регионе относительное высокое колебание находится в признаке f2. Учитывая, что в каждом регионе присутвуют нулевые значения показателя product, нужно посчитать их кол-во

In [12]:
display(len(data_0[data_0['product']== 0]))
display(len(data_1[data_1['product']== 0]))
display(len(data_2[data_2['product']== 0]))

1

8235

1

In [13]:
data_1[data_1['product'] == 0]

,f0,f1,f2,product
11,16.32,-0.56,-0.00,0.00
13,6.70,-0.75,-0.01,0.00
62,21.42,-5.13,-0.00,0.00
63,6.82,3.10,-0.00,0.00
77,6.75,-11.89,-0.00,0.00
...,...,...,...,...
99936,5.09,-3.98,0.01,0.00
99948,8.28,-9.18,0.00,0.00
99956,13.34,-1.29,0.01,0.00
99961,13.85,-11.53,-0.01,0.00


Получается, в 1 и 3 регионе по одному нулевому значению, а вот во втором регионе порядка 8 % (8200 с нулевым результом), что является большим значением,т.е. в реальном проекте нужно было бы сделать запрос с уточнением, а так будем считать данные синтетическими, а эти скважины неудачными. Удалять их не буду

Изучили данные, сеты готовы к обучению и проверки модели

## Обучение и проверка модели

Построим отдельные модели для каждого региона

Модель для первого региона

разделим данные на обучающую и валидационную выборки в соотношении

In [14]:
target_0 = data_0['product']
features_0 = data_0.drop(['product'], axis=1)

In [15]:
features_0_train, features_0_valid, target_0_train, target_0_valid = train_test_split(features_0,target_0, test_size=0.25,
                                                                              random_state=RANDOM_STATE)

In [16]:
features_0_train.shape, features_0_valid.shape

((74985, 3), (24995, 3))

масштабируем данные

In [17]:
scaler = StandardScaler()
scaler.fit(features_0_train)
features_0_train_log = scaler.transform(features_0_train)
features_0_valid_log = scaler.transform(features_0_valid)

In [18]:
display(features_0_train_log)
display(features_0_valid_log)

array([[ 1.34293395, -1.18475144, -0.82531087],
       [-0.28281145, -0.69664891, -1.50396313],
       [ 1.62246497, -0.11580161,  0.05182142],
       ...,
       [ 0.94632018, -1.353858  ,  0.19862366],
       [ 1.64769516,  0.25181381, -0.69400681],
       [-0.03913908, -0.99815247, -1.05910403]])

array([[-0.4013731 ,  0.53022612, -1.83865219],
       [-0.28668011, -0.90155086, -1.43705479],
       [ 1.40574652, -0.99213796, -1.44855272],
       ...,
       [-0.49434335, -0.64954289,  1.0855156 ],
       [ 1.62519377, -0.75174084,  0.31033546],
       [-1.0429708 ,  1.32901097, -1.09951378]])

In [19]:
model_0 = LinearRegression()
model_0.fit(features_0_train_log, target_0_train)

predictions_0 = model_0.predict(features_0_valid_log)


In [20]:
rmse_0 = (mean_squared_error(target_0_valid,predictions_0))**(0.5)
print("RMSE в первом регионе: {0:.2f}".format(rmse_0))

RMSE в первом регионе: 37.50


In [21]:
average_product_0 = sum(predictions_0) / len(predictions_0)
print('средний запас в первом регионе = {0:.2f}'.format(average_product_0))

средний запас в первом регионе = 92.58


Модель для второго региона

In [22]:
target_1 = data_1['product']
features_1 = data_1.drop(['product'], axis=1)

In [23]:
features_1_train, features_1_valid, target_1_train, target_1_valid = train_test_split(features_1,target_1, test_size=0.25,
                                                                              random_state=RANDOM_STATE)

In [24]:
features_1_train.shape, features_1_valid.shape

((74994, 3), (24998, 3))

In [25]:
scaler = StandardScaler()
scaler.fit(features_1_train)
features_1_train_log = scaler.transform(features_1_train)
features_1_valid_log = scaler.transform(features_1_valid)

In [26]:
display(features_1_train_log)
display(features_1_valid_log)

array([[ 0.94649269,  1.11626351, -0.29485436],
       [ 0.2234139 , -1.30934394,  0.29330896],
       [-0.8567036 , -0.32400061,  0.88155334],
       ...,
       [ 0.71996125,  2.21949477,  0.8744744 ],
       [-0.55832767, -0.17402487,  0.88154834],
       [ 1.18347265,  0.03167597, -0.87867228]])

array([[-0.53079814, -0.35323431, -0.29555687],
       [-0.72119892,  1.25592576, -0.29370633],
       [ 0.57102908,  1.09164422,  1.4663005 ],
       ...,
       [-0.68682694, -0.27132609,  1.47082804],
       [-1.31334743, -0.46670457, -1.47001836],
       [ 0.94576153, -0.10980442, -0.29480847]])

In [27]:
model_1 = LinearRegression()
model_1.fit(features_1_train_log, target_1_train)

predictions_1 = model_1.predict(features_1_valid_log)

In [28]:
rmse_1 = (mean_squared_error(target_1_valid,predictions_1))**(0.5)
print("RMSE во втором регионе: {0:.2f}".format(rmse_1))

RMSE во втором регионе: 0.89


In [29]:
average_product_1 = sum(predictions_1) / len(predictions_1)
print('средний запас во втором регионе = {0:.2f}'.format(average_product_1))

средний запас во втором регионе = 68.26


Модель для третьего региона

In [30]:
target_2 = data_2['product']
features_2 = data_2.drop(['product'], axis=1)

In [31]:
features_2_train, features_2_valid, target_2_train, target_2_valid = train_test_split(features_2,target_2, test_size=0.25,
                                                                              random_state=RANDOM_STATE)

In [32]:
features_2_train.shape, features_2_valid.shape

((74994, 3), (24998, 3))

In [33]:
scaler = StandardScaler()
scaler.fit(features_2_train)
features_2_train_log = scaler.transform(features_2_train)
features_2_valid_log = scaler.transform(features_2_valid)

In [34]:
display(features_2_train_log)
display(features_2_valid_log)

array([[-0.30280399, -1.53283556, -0.13436874],
       [-1.36168451, -0.97380354,  1.00363331],
       [-1.80373105,  1.55375937, -0.92968949],
       ...,
       [ 2.54242578,  1.57196785,  0.99332778],
       [ 0.36291832,  1.69846172,  0.87899602],
       [-0.02912389, -0.02071165, -0.73722049]])

array([[ 0.11882209,  0.60336689, -0.54946504],
       [-0.13687352,  0.52099516, -0.69961145],
       [ 0.37695551, -0.30070922, -1.70157466],
       ...,
       [ 1.62757972,  0.27207529,  0.16195985],
       [ 0.01732222, -0.04611174,  1.80081523],
       [-0.9300396 , -0.27701571,  1.11479081]])

In [35]:
model_2 = LinearRegression()
model_2.fit(features_2_train_log, target_2_train)

predictions_2 = model_2.predict(features_2_valid_log)

In [36]:
rmse_2 = (mean_squared_error(target_2_valid,predictions_2))**(0.5)
print("RMSE в третьем регионе: {0:.2f}".format(rmse_2))

RMSE в третьем регионе: 39.99


In [37]:
average_product_2 = sum(predictions_2) / len(predictions_2)
print('средний запас во втором регионе = {0:.2f}'.format(average_product_2))

средний запас во втором регионе = 95.13


Для наглядности и простоты сравнения моделей, собирем даные по обучению моделей в таблицу

In [38]:
results = pd.DataFrame(columns=(['region', 'rmse', 'avg']))
results.region=['регион 1', 'регион 2', 'регион 3']
results.rmse=[rmse_0, rmse_1, rmse_2]
results.avg=[average_product_0,average_product_1,average_product_2]
display(results) 

,region,rmse,avg
0,регион 1,37.50,92.58
1,регион 2,0.89,68.26
2,регион 3,39.99,95.13


<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b>если я правильно понял, то поставил переменные
</div>

по итогу данного наблюдения:
1) учитывая задание, использовали модель линейной регрессии, т.к. остальные не так предсказуемы

2) Лучший показатель rmse, практически равный 1, во втором регионе, что возможно связано с нулевыми значениями или переобучением модели, на втором месте 1 регион и на последнем 3.

3) Лучший показатель по средним запасам в третьем регионе,второе место- регион 1 и на последнем месте 2 регион


## Подготовка к расчёту прибыли

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

Бюджет на разработку скважин в регионе — 10 млрд рублей.

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

In [39]:
objects_for_study = 500
the_best_objects =200
budget_per_region = 10**10
revenue_per_barrel = 450*1000
probability_of_loss = 0.025

In [40]:
breakeven_rate = (budget_per_region/ revenue_per_barrel)/(the_best_objects)
print(f'Точка безубыточности объема запаса скважины на 200 объектов:  {breakeven_rate} ')

Точка безубыточности объема запаса скважины на 200 объектов:  111.11111111111111 


In [41]:
breakeven_rate_region = (budget_per_region/ revenue_per_barrel)
print(f'Точка безубыточности объема запаса региона:  {breakeven_rate_region} ')

Точка безубыточности объема запаса региона:  22222.222222222223 


Получается, что точка безубыточности на скважину составляет 111, точка безубыточности по региону 22 222. Учитывая, средние показатели полученые в модели, второй региона выглядит менее интересно для инвестиций, 1 и 3 регион в среднем ниже, но из данных видно, что порядка 25 % находятся в зоне безубыточности

## Расчёт прибыли и рисков 

In [42]:
def revenue(target_true, target_pred):
    best_pred_max200 = pd.Series(target_pred).sort_values(ascending=False)[:the_best_objects]
    best_true_max200 = (target_true.reset_index(drop = True)[best_pred_max200.index])
    best_sum =  best_true_max200.sum()
    return round((best_sum * revenue_per_barrel) - budget_per_region, 2)

In [43]:
print('Прибыль 200 скважин в регионе 1:', revenue(target_0_valid, predictions_0))
print('Прибыль 200 скважин в регионе 2:', revenue(target_1_valid, predictions_1))
print('Прибыль 200 скважин в регионе 3:', revenue(target_2_valid, predictions_2))

Прибыль 200 скважин в регионе 1: 3265148771.92
Прибыль 200 скважин в регионе 2: 2415086696.68
Прибыль 200 скважин в регионе 3: 2451128929.38


In [44]:
def bootstrap(target_true, target_pred):
    samples = []
    for i in range(1000):
        target_sample = pd.Series(target_pred).sample(n = objects_for_study, replace=True, random_state = RANDOM_STATE)
        samples.append(revenue(target_true,target_sample))
    samples = pd.Series(samples)
    lower = samples.quantile(0.025)
    upper = samples.quantile(0.975)
    print('Средняя прибыль 200 скважин:', samples.mean())
    print('Риск убытков:', samples.apply(lambda x: x < 0).sum()/len(samples)*100,"%")
    print("5% доверительный итервал для региона  :",round(lower,2), round(upper,2))



In [45]:
print(bootstrap(target_0_valid,predictions_0))



Средняя прибыль 200 скважин: 448711699.35847
Риск убытков: 4.7 %
5% доверительный итервал для региона  : -55785826.13 972593754.26
None


In [46]:
print(bootstrap(target_1_valid, predictions_1))

Средняя прибыль 200 скважин: 385851350.08052003
Риск убытков: 2.3 %
5% доверительный итервал для региона  : 13042655.14 792330369.71
None


In [47]:
print(bootstrap(target_2_valid, predictions_2))

Средняя прибыль 200 скважин: 406243019.08451
Риск убытков: 6.4 %
5% доверительный итервал для региона  : -115274352.33 903934171.16
None


Вывод:

     Согласно заданию, нужно выбрать регион  с риском убытков менее 2,5 %. у нас получается только один регион соответствует данному критерию и это регион номер 2. стоит отметить, что доверительнй интервалу данного региона находится в положительной зоне и разброс по нему является наименьшим. при этом средняя прибыль по 200 точкам в данном регионе является наименьшей среди рассматриваемых 3 регионов. на первом месте по прибыли и с риском убытков на второй позиции, но выше заданных критереев, находится регион 1.

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [ ]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован